In [1]:
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import time
import glob
from lxml.html import parse
from sphfile import SPHFile
import pydub
import audiosegment
import pandas as pd
from collections import Counter
from bs4 import BeautifulSoup
import sys

In [2]:
start_df = pd.read_csv('trainfiles/start.csv')
end_df = pd.read_csv('trainfiles/end.csv')

In [3]:
segments_df = start_df.append(end_df)
speakers = Counter(segments_df['speaker'])
common_speakers = speakers.most_common(20)
segments_by_speakers = {speaker:segments_df[segments_df['speaker']==speaker] for speaker, count in common_speakers}
segments_by_speakers
common_speakers = [common_speaker[0] for common_speaker in common_speakers]

In [4]:
overlay = []
for first_speaker in common_speakers:
    second_speakers = [speaker for speaker in common_speakers if speaker!=first_speaker ]
    first_segments = segments_by_speakers[first_speaker]
    for second_speaker in second_speakers:
        second_segments = segments_by_speakers[second_speaker]
        first_idx = np.arange(290)
        second_idx = np.arange(290)
        np.random.shuffle(first_idx)
        np.random.shuffle(second_idx)
        for i in range(290):
            first_segment = first_segments.iloc[first_idx[i]]
            second_segment = second_segments.iloc[second_idx[i]]
            row = {'first_speaker': first_speaker, 'second_speaker': second_speaker,
                  'first_file': first_segment['segfile'], 'second_file': second_segment['segfile']}
            overlay.append(row)
overlay = pd.DataFrame(overlay)

In [6]:
overlay_shuffled = overlay.sample(frac=1)
tenth = int(len(overlay)*0.9)
overlay_shuffled[:8*tenth].to_csv('overlay-train.csv')
overlay_shuffled[8:tenth:9*tenth].to_csv('overlay-val.csv')
overlay_shuffled[9*tenth:].to_csv('overlay-test.csv')